# Machine Learning avancé : Deep Learning for NLP
On va mettre au point nos premiers modèles Deep Learning appliqué au traitement du langage. Plutôt que de tout recoder de zéro, on va ici s'appuyer sur la puissance de la librairie Keras qui nous permettra de connecter les layers à la volée et d'implémenter des architectures plus exotiques.

Après cela vous saurez:
- utiliser un embedding pré-calculé
- construire un réseau de neurones avec Keras
- construire une architecture custom avec Keras

Cet exercice constitue également le test noté du cours

## Import des Librairies

In [29]:
import data_utils.utils as du
import data_utils.pos as pos 
import numpy as np
import pandas
data='data/'
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, Flatten, Input, Masking
from keras.optimizers import RMSprop
from keras import backend as K

## Présentation du problème
Le POS-Tagging et le Shallow Parsing constituent deux tâches classiques en NLP :
- POS-Tagging : affecte à chaque mot un tag unique qui indique son rôle syntaxique (nom, verbe, adverbe, ..)
- Shallow Parsing : affecte à chaque segment de phrase un tag unique qui indique le rôle de l'élément syntaxique auquel il appartient (groupe nominal, groupe verbal, etc..)
Les fichiers sont ici au format ConLL : un mot par ligne, les phrases sont séparées par un saut de ligne.

Nous allons ici refaire les travaux de l'article NLP almost from scratch, qui consiste à créer un réseau de neurones pour effectuer chaque tâche, puis nous ferons un modèle partagé et enfin un modèle hiearchique.

<img src="MLP.png" style="width:300px;height:450px;">
<caption><center> <u>Figure 1</u>: Modèle simple pour les tâches POS-Tagging et Shallow Parsing</center></caption>

# POS-Tagging
## Import des données

A l'aide des fonctions d'aide, importer les données :

#### Word embedding

In [30]:
wv, word_to_num, num_to_word = pos.load_wv(
      data+'vocab.txt', data+'wordVectors.txt')
print('wordvector shape is',wv.shape)

wordvector shape is (100003, 50)


On récupère également les tags et on crée les dictionnaires adéquats

#### Création des tags

In [31]:
tagnames = ['ADJ','ADJWH','ADV','ADVWH','CC','CLO',
                'CLR','CLS','CS','DET','DETWH','ET','I','NC',
                'NPP','P','P+D','P+PRO','PONCT','PREF','PRO',
                'PROREL','PROWH','VINF','VPR','VPP','V','VS','VIMP']
num_to_tag = dict(enumerate(tagnames))
tag_to_num = {v:k for k,v in num_to_tag.items()}

#### Chargement des données text

Chargement des documents puis création des matrices X_train, y_train, X_test, y_test
le paramètre wsize précise la taille de la fenêtre, choisissez une valeur par défaut parmis (3,5,7)

In [32]:
docs_train = du.load_dataset(data+'train.txt') # liste qui contient les phrases et les tags
X_train, y_train = du.docs_to_windows(docs_train, word_to_num, tag_to_num, wsize=3) # parcourt la liste et créer les matrices 

In [33]:
docs_train[0]

[['paul', 'NC'],
 ['jules', 'ADJ'],
 ['antoine', 'NC'],
 ['meillet', 'ADJ'],
 [',', 'PONCT'],
 ['né', 'VPP'],
 ['le', 'DET'],
 ['à', 'P'],
 ['moulins', 'NC'],
 ['(', 'PONCT'],
 ['allier', 'NC'],
 [')', 'PONCT'],
 ['et', 'CC'],
 ['mort', 'ADV'],
 ['le', '_ADV'],
 ['à', '_ADV'],
 ['châteaumeillant', '_ADV'],
 ['(', 'PONCT'],
 ['cher', 'ADJ'],
 [')', 'PONCT'],
 [',', 'PONCT'],
 ['est', 'V'],
 ['le', 'DET'],
 ['principal', 'ADJ'],
 ['linguiste', 'NC'],
 ['français', 'ADJ'],
 ['des', 'P+D'],
 ['premières', 'ADJ'],
 ['décennies', 'NC'],
 ['du', 'P+D'],
 ['.', 'PONCT']]

In [34]:
docs_test = du.load_dataset(data+'test.txt')
X_test, y_test = du.docs_to_windows(
    docs_test, word_to_num, tag_to_num, wsize=3)

In [35]:
y_train = keras.utils.to_categorical(y_train, 29)
y_test = keras.utils.to_categorical(y_test, 29)

On extrait les lignes dans y_test qui contiennent des mots non-présents dans X_train : out of vocabulary.

In [36]:
X_test_oov,Y_test_oov = du.get_oov(X_train,y_train,X_test,y_test)

In [37]:
print("X_train a pour dimension",X_train.shape)
print("X_test a pour dimension",X_test.shape)
print("y_train a pour dimension",y_train.shape)
print("y_test a pour dimension",y_test.shape)

X_train a pour dimension (680238, 3)
X_test a pour dimension (267334, 3)
y_train a pour dimension (680238, 29)
y_test a pour dimension (267334, 29)


### Question :

Expliquer ce que contient X et Y ci-dessus ainsi que leur dimension. (NB: Lorsque l'on définit une fenêtre on est amené à créer un tag pour le début et la fin de phrase afin que les premiers et derniers mots puissent être considérés dans le modèle

### Réponse :

X_train et X_test contiennent les fenêtres de mots qui seront utilisées comme entrées pour le modèle. Chaque fenêtre est constituée de trois mots, où le mot central est l'étiquette à prédire et les deux mots voisins sont les contextes. Les dimensions de X_train et X_test signifie qu'il y a 680238 et 267334 fenêtres dans les ensembles d'entraînement et de test, respectivement, chacune avec trois mots.

y_train et y_test contiennent les étiquettes. Les dimensions de y_train et y_test signifie qu'il y a 680238 et 267334 fenêtres dans les ensembles d'entraînement et de test, respectivement, chacune avec une étiquette correspondant à l'une des 29 classes de POS tag.

## Création du NN

### Consigne :
Compléter le code ci-dessous pour définir une architecture :
Embed (dim 50) -> Dense -> Dropout -> Predict (Softmax).
N'hésitez pas à consulter l'aide de Keras pour :
    - models.Sequential
    - layers.Embedding
    - layers.Flatten
NB : Prendre garde à bien laisser l'embedding "entrainable" afin que la représentation vectorielle bénéficie aussi de la backprop

In [41]:
model = Sequential()
# Your code Here#
model.add(Embedding(input_dim=len(word_to_num), output_dim=50, input_length=3, trainable=True))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(29, activation='softmax'))

## Entrainement

### Questions :
- A quoi servent les layers suivants :
    - Flatten()
    - Dropout()
- Combien de paramètres seront appris au cours de l'entrainement? (il existe une commande qui permette de trouver l'architecture du réseau de neurones)

### Réponse
Flatten() est utilisé pour convertir la sortie du précédent Dense layer en un vecteur 1D pour qu'elle puisse être utilisée par la couche suivante.

Dropout() est une méthode de régularisation pour prévenir le overfitting en réduisant les effets de la coadaptation des neurones. Elle éteint aléatoirement une proportion des neurones d'entrée à chaque étape de l'entraînement.

Le nombre de paramètres est de 5000150 pour l'embedding, 3264 pour la couche dense, et 11165 pour la dernière couche dense total = 5.014.579 parametres

### Consigne : Compilation
Compléter le code ci-dessous en choisissant l'optimizer RMSprop, et en choissisant la bonne loss (NB on est sur un probléme de classification à 29 modalités). La métrique sera renvoyée par les logs au cours de l'entrainement.

In [42]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

### Consigne  : Entrainement
Compléter le code ci-dessous pour entrainer sur le couple X_train, Y_train et en validant sur X_test, Y_test. Vous devriez pouvoir atteindre une accuracy de 93% au bout de 3 itérations. N'oubliez pas que la taille du batch correspond au nombre d'exemples utilisés pour estimer le gradient. Il peut influer sur la convergence.

In [43]:
history = model.fit(X_train, y_train,
                    epochs=3,
                    batch_size=128,
                    validation_data=(X_test, y_test))

Epoch 1/3
5315/5315 [==============================] - 76s 14ms/step - loss: 0.4986 - accuracy: 0.8657 - val_loss: 0.2969 - val_accuracy: 0.9163
Epoch 2/3
5315/5315 [==============================] - 75s 14ms/step - loss: 0.3135 - accuracy: 0.9150 - val_loss: 0.2710 - val_accuracy: 0.9234
Epoch 3/3
5315/5315 [==============================] - 92s 17ms/step - loss: 0.2933 - accuracy: 0.9210 - val_loss: 0.2647 - val_accuracy: 0.9258


### Question :
- Nous avons décidé de regarder l'accuracy comme métrique. Expliquez en quoi ce choix est discutable.
- On se propose de tester la performance sur des données qui contiennent des mots non-vus lors du train dans la cellule ci-dessous lancer l'évaluation sur X_test_oov,Y_test_oov

### Réponse :

Elle peut être trompeuse, par exemple avec des classes déséquilibrées. On pourrait regarder la précision, le rappel ou le F1-score. Sinon pour les problèmes des imbalaced classes, la bacc (balance accuracy)

In [48]:
score = model.evaluate(X_test_oov, Y_test_oov)
print(score[1])

779/779 [==============================] - 1s 1ms/step - loss: 0.4421 - accuracy: 0.8713
0.871318519115448


## Consigne :
- Déterminer quelle est la meilleur taille de fenêtre selon vous (tester les plusieurs hypothèses parmis 3, 5 et 7)
- Reprendre le code pour les différentes hypothèses suivantes avec la taille de fenêtre choisie ci-dessus :
    - 1 embedding random non entrainable
    - 2 embedding pré-entrainer (wv) entrainable
    - 3 embedding pré-entrainer (wv) non entrainable
- Conclure en testant sur X_test_oov et Y_test_oov :
    - Quelle est la méthode que vous choisiriez et pourquoi? 
    - Pouvez-vous expliquer intuitivement les différents résultats ? 

In [54]:
# Define window sizes to test
window_sizes = [3, 5, 7]

# Loop over window sizes
for wsize in window_sizes:
    
    print(f'Window: {wsize}')
    
    X_train, y_train = du.docs_to_windows(docs_train, word_to_num, tag_to_num, wsize=wsize)
    X_test, y_test = du.docs_to_windows(docs_test, word_to_num, tag_to_num, wsize=wsize)
    
    y_train = keras.utils.to_categorical(y_train, 29)
    y_test = keras.utils.to_categorical(y_test, 29)
    
    model = Sequential()
    model.add(Embedding(input_dim=len(word_to_num), output_dim=50, input_length=wsize, trainable=False))
    model.add(Dense(128, activation='relu'))
    model.add(Flatten())
    model.add(Dense(29, activation='softmax'))


    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train,
                        epochs=3,
                        batch_size=128,
                        validation_data=(X_test, y_test),
                        verbose=0)
    
    score = model.evaluate(X_test_oov, Y_test_oov, verbose=0)
    print(f'window size {wsize} accuracy,{score[1]}')


Window: 3
window size 3 accuracy,0.79913330078125
Window: 5


2023-03-29 22:48:21.668553: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: Matrix size-incompatible: In[0]: [32,384], In[1]: [640,29]
	 [[{{node sequential_13/dense_21/BiasAdd}}]]


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_13/dense_21/BiasAdd' defined at (most recent call last):
    File "/Users/pepegarcia/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/pepegarcia/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/pepegarcia/Documents/GitHub/ML_avance/venv/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/pepegarcia/Documents/GitHub/ML_avance/venv/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/Users/pepegarcia/Documents/GitHub/ML_avance/venv/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/Users/pepegarcia/Documents/GitHub/ML_avance/venv/lib/python3.9/site-packages/tornado/platform/a

Il est possible de sauvegarder le modèle (les poids W) dans un fichier .h5

In [50]:
model.save("./model")

2023-03-29 22:38:44.748961: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,3,50]
	 [[{{node inputs}}]]
2023-03-29 22:38:44.754927: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,3,64]
	 [[{{node inputs}}]]
2023-03-29 22:38:44.967337: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,3,50]
	 [[{{node inputs}}]]
2023-03-29 22

INFO:tensorflow:Assets written to: ./model/assets


# Shallow Parsing

## Consigne :
- Reprendre la méthodologie ci-dessus et entrainer un modèle de shallow parsing
- Les tags sont ['O','B-NP','I-NP','B-AP','I-AP','B-CONJ',
                'I-CONJ','B-AdP','I-AdP','B-VN','I-VN','B-PP','I-PP','B-UNKNOWN','I-UNKNOWN']
- Les fichiers sont train_chunk.txt et test_chunk.txt
- N'oubliez pas de recréer les tests out-of-vocabulary (X_test_oov et Y_test_oov)
- Conclure sur votre choix du meilleur modèle

#### Word Embedding

In [ ]:
# Load word vectors
wv, word_to_num, num_to_word = pos.load_wv(data+'vocab.txt', data+'wordVectors.txt')

# Load data
docs_train = du.load_dataset(data+'train_chunk.txt')
X_train, y_train = du.docs_to_windows(docs_train, word_to_num, tag_to_num, wsize=5)
docs_test = du.load_dataset(data+'test_chunk.txt')
X_test, y_test = du.docs_to_windows(docs_test, word_to_num, tag_to_num, wsize=5)

# Convert labels to categorical format
y_train = keras.utils.to_categorical(y_train, 15)
y_test = keras.utils.to_categorical(y_test, 15)

# Create out-of-vocabulary test set
X_test_oov, Y_test_oov = du.get_oov(X_train, y_train, X_test, y_test)

# Define model architecture
model = Sequential()
model.add(Embedding(input_dim=len(word_to_num), output_dim=50, input_length=5, trainable=True))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(15, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train,
                    epochs=3,
                    batch_size=128,
                    validation_data=(X_test, y_test))

# Evaluate model on out-of-vocabulary test set
score = model.evaluate(X_test_oov, Y_test_oov, verbose=0)
print('Test accuracy:', score[1])

# Save model weights to file
model.save('shallow_parser.h5')


#### Création des tags

#### Chargement des données

#### Création du réseau de neurones

#### Entrainement

#### Conclusion

# Multi-task learning

## Full multi tagged (POS tagging + Shallow parsing)

En deep learning, il est possible et parfois même recommandé d'entrainer un réseau de neurones sur plusieurs tâches en même temps. Intuitivement on se dit que l'apprentissage de représentation sur une tâche devrait pouvoir aider sur une autre tâche. Cela permet en outre de disposer de plus de données par exemple et/ou d'avoir un modèle plus robuste, plus précis.

Dans cette partie, on va s'appuyer sur la classe Model de Keras https://keras.io/getting-started/functional-api-guide/. Plutôt que d'ajouter les layers, il s'agit plutôt de voire chaque layer comme une fonction. Il s'agit alors d'enchainer les fonctions.

<img src="mtl_images.png" style="width:300px;height:450px;">
<caption><center> <u>Figure 1</u>: Un exemple d'architecture pour apprendre 3 tâches </center></caption>

#### Embedding

In [ ]:
wv, word_to_num, num_to_word = pos.load_wv(
      data+'vocab.txt', data+'wordVectors.txt')
print('wordvector shape is',wv.shape)

#### Création des tags

In [ ]:
postagnames = ['ADJ','ADJWH','ADV','ADVWH','CC','CLO',
                'CLR','CLS','CS','DET','DETWH','ET','I','NC',
                'NPP','P','P+D','P+PRO','PONCT','PREF','PRO',
                'PROREL','PROWH','VINF','VPR','VPP','V','VS','VIMP']
num_to_postag = dict(enumerate(postagnames))
postag_to_num = {v:k for k,v in num_to_postag.items()}

In [ ]:
chunktagnames = ['O','B-NP','I-NP','B-AP','I-AP','B-CONJ',
                'I-CONJ','B-AdP','I-AdP','B-VN','I-VN','B-PP','I-PP','B-UNKNOWN','I-UNKNOWN']
num_to_chunktag = dict(enumerate(chunktagnames))
chunktag_to_num = {v:k for k,v in num_to_chunktag.items()}

#### Import des données

In [ ]:
# Load the training set
docs = du.load_dataset(data+'train.txt')
X_train, pos_train = du.docs_to_windows(
    docs, word_to_num, postag_to_num, wsize=5)
docs = du.load_dataset(data+'train_chunk')
X_train, chunk_train = du.docs_to_windows(
    docs, word_to_num, chunktag_to_num, wsize=5)

In [ ]:
docs = du.load_dataset(data+'test.txt')
X_test, pos_test = du.docs_to_windows(
    docs, word_to_num, postag_to_num, wsize=5)
docs = du.load_dataset(data+'test_chunk')
X_test, chunk_test = du.docs_to_windows(
    docs, word_to_num, chunktag_to_num, wsize=5)

#### Mise en forme pour l'apprentissage

In [ ]:
X_pos_train=X_train[:400000,:]

X_chunk_train=X_train[-400000:,]

In [ ]:
Y_pos_train = keras.utils.to_categorical(pos_train[0:400000,], 29)
Y_pos_test = keras.utils.to_categorical(pos_test, 29)
Y_chunk_train = keras.utils.to_categorical(chunk_train, 15)
Y_chunk_test = keras.utils.to_categorical(chunk_test[-400000:,], 15)

#### Résumé
Jusqu'à présent on dispose de :
- wv qui contient un embedding
- X_pos_ et Y_pos_ qui contiennent respectivement les mots et le tag sur train et test pour le pos tagging
- X_chunk_ et Y_chunk_ qui contiennent respectivement les mots et le tag sur train et test pour le shallow parsing
- X_pos et X_chunk n'ont aucune fenêtre en commun

### Création du réseaux de neurones commun aux deux tâches

#### Consigne
Utiliser la même architecture que précedemment pour construire le réseau de neurones communs : shared_nn

In [ ]:
shared_nn=Sequential()

#### Consigne
Définir les inputs du réseaux de neurones communs à l'aide de la fonction Input. Ce sont les input layer de notre architecture, chacun correspond à une tâche.

In [ ]:
pos_input = 
chunk_input =

#### Consigne
- Définir pos_representation et chunk_representation comme les images respectibes de chaque input par shared_nn
- Définir pos_target et chunk_target comme l'image des représentations ci-dessus par des layers dense avec activation softmax (cf  la fonction Dense)

Rappel : la fonction softmax va calculer $n$ softmax, où $n$ est le nombre de classe en suivant la formule :
$$\sigma(z)_j = \frac{\exp(z_j)}{\sum_{i}\exp(z_i)},$$
avec $z$ le vecteur en sortie du réseau de neurones et $z_i$ sa $i$-ième composante

In [ ]:
pos_representation =
chunk_representation =

pos_target = 
chunk_target = 

### Entrainement et conclusion

#### Consigne 
Jusqu'ici on a juste défini la succession d'opération, il s'agit maintenant de créer le modèle en utilisant la classe modèle de Keras

In [ ]:
model= Model(inputs=,outputs=)

#### Consigne
- compiler le modèle (NB : il y a une loss pour chaque tâche)
- afficher l'architecture
- entrainer le modèle pour le nombre d'époque suffisant en validant sur X_test et Y_test et une taille de batch de 128
- Conclure en comparant avec les performances précédentes

In [ ]:
model.compile()

In [ ]:
model.fit()

## Hierarchical learning
Une autre façon de faire du multi-task consiste à construire une architecture en cascade où les tâches n'interviennent pas à la même profondeur du réseau de neurones.

### Consigne :
- En vous inspirant de la partie précedente, entrainer un modèle en cascade de type : 
                              POS
                            /
EMBEDDING - DENSE - DROPOUT 
                            \
                              DENSE - DROPOUT - CHUNK
                              
Autrement dit, dans cette approche, on pré-suppose que  
- EMBEDDING - DENSE - DROPOUT apprend une représentation suffisante pour prédire le POS-TAG
- plus de layer sont nécessaires pour le Shallow parsing
- la représentation intermédiaire EMBEDDING - DENSE - DROPOUT est un bon point de départ pour le shallow parsing

# Question optionnelle 
- Comment feriez vous, en utilisant la classe Model, pour pouvoir extraire l'embedding tuné sur le modèle pour un mot donné? Tenter de l'implémenter.